In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import pickle

In [2]:
#Lode data
test_set = pd.read_excel(r'C:\Master\IS\train_set.xlsx')
test_set.head()

col_detail = test_set['Detail'].tolist()

In [3]:
#Clean-data
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words

import re
import string
import nltk
nltk.download("stopwords")
th_stop = thai_stopwords()
en_stop = tuple(get_stop_words('en'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def clean_msg(msg):
    
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

In [5]:
def split_word(text):
            
    
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    tokens = [i for i in tokens if not i in th_stop and not i in en_stop]
    
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

In [6]:
clean_text = [clean_msg(txt) for txt in col_detail]
tokens_list = [split_word(txt) for txt in clean_text]

In [7]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from pythainlp.corpus import thai_stopwords
stop_words= thai_stopwords()

In [8]:
def identity_fun(text):
    return text

In [9]:
tfidf_vectorizer = TfidfVectorizer(analyzer = 'word', #this is default
                                   tokenizer=identity_fun, #does no extra tokenizing
                                   preprocessor=identity_fun, #no extra preprocessor
                                   token_pattern=None)

In [10]:
tfidf_vector= tfidf_vectorizer.fit_transform(tokens_list)
tfidf_array = np.array(tfidf_vector.todense())

#แปลงเป็น DataFrame เพื่อง่ายแก่การอ่าน
df = pd.DataFrame(tfidf_array,columns=tfidf_vectorizer.get_feature_names_out())
df

,A,AIA,ASAP,ATM,Abdomen,Access,Admit,AdultPed,Aetna,Affordable,...,ๆๆๆๆๆๆส์,ๆ​,ๆ”,์,์คง,์ฯ,​,’,“,”
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
pickle.dump(tfidf_vectorizer, open('tfidf_vectorizer.sav','wb'))

In [12]:
#ML Model
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [13]:
answer  = test_set['Class']
len(answer)

5885

In [14]:
X = df
y = answer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
#Create a svm Classifier
clf = SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [16]:
print(classification_report(y_test, y_pred, digits = 6))

              precision    recall  f1-score   support

           0   0.968379  0.959843  0.964092      1021
           1   0.945623  0.957047  0.951301       745

    accuracy                       0.958664      1766
   macro avg   0.957001  0.958445  0.957697      1766
weighted avg   0.958780  0.958664  0.958696      1766



In [17]:
confusion_matrix(y_test, y_pred)

array([[980,  41],
       [ 32, 713]], dtype=int64)

In [18]:
pickle.dump(clf, open('svm.sav','wb'))

In [19]:
#Blind
blind_test = pd.read_excel(r'C:\Master\IS\blind_test.xlsx')
blind_test.head()

,Unnamed: 0,Class,Detail,Department,Aspect
0,6544,1,จอดรถไกล,ENV,5
1,1885,1,อยากให้ลิฟต์สำหรับผู้ป่วยแยก เวลาขึ้นลงลำบาก ท...,ENV,7
2,4301,0,เจ้าหน้าที่ยิ้มแย้มดีค่ะ,EX5WEL,0
3,5073,1,อยากให้จัดสรรที่จอดรถ ไม่เพียงพอ มีน้อยมาก,ENV,5
4,5720,0,บริการดี,OPD12ENT,1


In [20]:
col_detail2 = blind_test['Detail'].tolist()

In [21]:
len(col_detail2)

1038

In [22]:
clean_text2 = [clean_msg(txt) for txt in col_detail2]

In [23]:
tokens_list2 = [split_word(txt) for txt in clean_text2]
tokens_list2

[['จอด', 'รถ'],
 ['ลิฟต์', 'สำหรับ', 'ผู้ป่วย', 'แยก', 'เวลา', 'ลำบาก', 'ที่จอดรถ', 'ลำบาก'],
 ['เจ้าหน้าที่', 'ยิ้มแย้ม', 'ดี'],
 ['จัดสรร', 'ที่จอดรถ'],
 ['บริการ', 'ดี'],
 ['ค่า', 'จอด', 'รถ', 'โรงแรม'],
 ['เภสัชกร',
  'ประจำ',
  'ห้อง',
  'ยา',
  'ชั้น',
  'อาคาร',
  'ศูนย์',
  'ความเป็นเลิศ',
  'ทางการแพทย์',
  'รับโทรศัพท์',
  'ผู้รับ',
  'บริการ',
  'แจ้ง',
  'ยา',
  'วันที่',
  'กรกฎาคม',
  'จำนวน',
  'รายการ',
  'Meteospasmyl',
  'KremilS',
  'แพทย์',
  'สั่ง',
  'จำนวน',
  'เม็ด',
  'ผู้รับ',
  'บริการ',
  'เม็ด',
  'ประสงค์',
  'ที่จะ',
  'แจ้งให้ทราบ',
  'ยา',
  'ระมัดระวัง',
  'มากขึ้น'],
 ['ผ้าม่าน',
  'ห้องน้ำ',
  'เชื้อรา',
  'ทำความสะอาด',
  'ชั้น',
  'อาคาร',
  'ศรี',
  'พัฒน',
  '์'],
 ['รักษา', 'ดีมาก'],
 ['S', 'A', 'พยาบาล', 'ดูแล', 'ดี', 'คุณหมอ', 'ใจ', 'ดีมาก', 'อก', 'ฤช'],
 ['แพทย์',
  'คำแนะนำ',
  'ดีมาก',
  'พูดจา',
  'ไพเราะ',
  'พยาบาล',
  'ผู้ช่วยพยาบาล',
  'ให้บริการ',
  'ดีมาก',
  'พูดจา',
  'ไพเราะ',
  'มารยาท',
  'ดีมาก',
  'น้องๆ',
  'ความช่วยเหลือ',
 

In [24]:
tfidf_vector2= tfidf_vectorizer.transform(tokens_list2)
tfidf_array2 = np.array(tfidf_vector2.todense())

#แปลงเป็น DataFrame เพื่อง่ายแก่การอ่าน
df2 = pd.DataFrame(tfidf_array2,columns=tfidf_vectorizer.get_feature_names())
df2

C:\Users\Acer\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,A,AIA,ASAP,ATM,Abdomen,Access,Admit,AdultPed,Aetna,Affordable,...,ๆๆๆๆๆๆส์,ๆ​,ๆ”,์,์คง,์ฯ,​,’,“,”
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
a = ('[บริการดีมากพนักงานนิสัยดีอยากกลับมาใช้ซ้ำ]',)

In [26]:
clean_text3 = [clean_msg(txt) for txt in a]
tokens_list3 = [split_word(txt) for txt in clean_text3]
tokens_list3

[['บริการ', 'ดีมาก', 'พนักงาน', 'นิสัย', 'ดี', 'กลับมา', 'ซ้ำ']]

In [27]:
tfidf_vector3= tfidf_vectorizer.transform(tokens_list3)
tfidf_array3 = np.array(tfidf_vector3.todense())

#แปลงเป็น DataFrame เพื่อง่ายแก่การอ่าน
df3 = pd.DataFrame(tfidf_array3,columns=tfidf_vectorizer.get_feature_names())
df3

C:\Users\Acer\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,A,AIA,ASAP,ATM,Abdomen,Access,Admit,AdultPed,Aetna,Affordable,...,ๆๆๆๆๆๆส์,ๆ​,ๆ”,์,์คง,์ฯ,​,’,“,”
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
y_pred3 = clf.predict(df3)
y_pred3

array([0], dtype=int64)

In [29]:
X = df
Y = test_set['Aspect']

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [31]:
from sklearn.linear_model import SGDClassifier

In [49]:
SGD = SGDClassifier(loss="hinge", penalty="l2", max_iter=500)
SGD.fit(X_train, Y_train)
Y_pred_SGD = SGD.predict(X_test)
print(classification_report(Y_test, Y_pred_SGD, digits = 6))
confusion_matrix(Y_test, Y_pred_SGD)

              precision    recall  f1-score   support

           0   0.677596  0.514523  0.584906       241
           1   0.781940  0.890591  0.832737       914
           2   0.696970  0.766667  0.730159        30
           3   0.561644  0.401961  0.468571       102
           4   0.000000  0.000000  0.000000         3
           5   0.894515  0.824903  0.858300       257
           6   0.689655  0.540541  0.606061        37
           7   0.829412  0.824561  0.826979       171
           8   0.000000  0.000000  0.000000        11

    accuracy                       0.778596      1766
   macro avg   0.570192  0.529305  0.545301      1766
weighted avg   0.766380  0.778596  0.767952      1766



C:\Users\Acer\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Acer\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Acer\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[124, 106,   1,   5,   0,   2,   0,   3,   0],
       [ 49, 814,   6,  26,   0,   8,   4,   7,   0],
       [  0,   3,  23,   0,   0,   1,   1,   2,   0],
       [  6,  51,   0,  41,   0,   3,   1,   0,   0],
       [  0,   2,   0,   0,   0,   0,   1,   0,   0],
       [  0,  28,   3,   1,   0, 212,   0,  13,   0],
       [  0,  12,   0,   0,   0,   2,  20,   3,   0],
       [  3,  17,   0,   0,   0,   8,   2, 141,   0],
       [  1,   8,   0,   0,   0,   1,   0,   1,   0]], dtype=int64)

In [52]:
pickle.dump(SGD, open('SGD.sav','wb'))

In [40]:
b = ('ได้รับการดูเลดีมาก ทุกอย่างน่าประทับใจ','ที่จอดรถแย่มาก ไม่มีที่ให้จอดเลย รอรถรับส่งนานมากๆ','คุณหมอเก่งมาก มีประสบการณ์รักษาพยาบาลเยอะมาก','ค่ารักษาพยาบาลถูกมาก','อาหารที่ได้รับไม่อร่อยจืดมาก',"ไม่ค่อยได้รับข้อมูลที่จำเป็นเลย","ประทับใจ อาจกลับมา")
b

('ได้รับการดูเลดีมาก ทุกอย่างน่าประทับใจ',
 'ที่จอดรถแย่มาก ไม่มีที่ให้จอดเลย รอรถรับส่งนานมากๆ',
 'คุณหมอเก่งมาก มีประสบการณ์รักษาพยาบาลเยอะมาก',
 'ค่ารักษาพยาบาลถูกมาก',
 'อาหารที่ได้รับไม่อร่อยจืดมาก',
 'ไม่ค่อยได้รับข้อมูลที่จำเป็นเลย',
 'ประทับใจ อาจกลับมา')

In [41]:
clean_text4 = [clean_msg(txt) for txt in b]
tokens_list4 = [split_word(txt) for txt in clean_text4]
tokens_list4

[['ดู', 'เล', 'ดีมาก', 'น่าประทับใจ'],
 ['ที่จอดรถ', 'แย่มาก', 'จอด', 'รอ', 'รถ'],
 ['คุณหมอ', 'เก่ง', 'มีประสบการณ์', 'รักษาพยาบาล'],
 ['ค่ารักษาพยาบาล'],
 ['อาหาร', 'อร่อย', 'จืด'],
 ['ข้อมูล'],
 ['ประทับใจ', 'กลับมา']]

In [45]:
tfidf_vector4= tfidf_vectorizer.transform(tokens_list4)
tfidf_array4 = np.array(tfidf_vector4.todense())

#แปลงเป็น DataFrame เพื่อง่ายแก่การอ่าน
df4 = pd.DataFrame(tfidf_array4,columns=tfidf_vectorizer.get_feature_names())
df4

C:\Users\Acer\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,A,AIA,ASAP,ATM,Abdomen,Access,Admit,AdultPed,Aetna,Affordable,...,ๆๆๆๆๆๆส์,ๆ​,ๆ”,์,์คง,์ฯ,​,’,“,”
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
y_pred5 = SGD.predict(df4)
y_pred5

array([1, 5, 1, 1, 2, 1, 1], dtype=int64)